In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.contrib.eager as tfe

import utils
import word2vec_utils

tfe.enable_eager_execution()

In [2]:
# Model hyperparameters
VOCAB_SIZE = 50000
BATCH_SIZE = 128
EMBED_SIZE = 128            # dimension of the word embedding vectors
SKIP_WINDOW = 1             # the context window
NUM_SAMPLED = 64            # number of negative examples to sample
LEARNING_RATE = 1.0
NUM_TRAIN_STEPS = 100000
VISUAL_FLD = 'visualization'
SKIP_STEP = 5000

In [3]:
# Parameters for downloading data
DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016

In [4]:
class Word2Vec(object):
  def __init__(self, vocab_size, embed_size, num_sampled=NUM_SAMPLED):
    self.vocab_size = vocab_size
    self.num_sampled = num_sampled
    self.embed_matrix = tfe.Variable(tf.random_uniform(
                                      [vocab_size, embed_size]))
    self.nce_weight = tfe.Variable(tf.truncated_normal(
                                    [vocab_size, embed_size],
                                    stddev=1.0 / (embed_size ** 0.5)))
    self.nce_bias = tfe.Variable(tf.zeros([vocab_size]))

  def compute_loss(self, center_words, target_words):
    """Computes the forward pass of word2vec with the NCE loss.""" 
    embed = tf.nn.embedding_lookup(self.embed_matrix, center_words)
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=self.nce_weight, 
                                        biases=self.nce_bias, 
                                        labels=target_words, 
                                        inputs=embed, 
                                        num_sampled=self.num_sampled, 
                                        num_classes=self.vocab_size))
    return loss

In [5]:
def gen():
  yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES,
                                      VOCAB_SIZE, BATCH_SIZE, SKIP_WINDOW,
                                      VISUAL_FLD)

In [6]:
def main():
  dataset = tf.data.Dataset.from_generator(gen, (tf.int32, tf.int32),
                              (tf.TensorShape([BATCH_SIZE]),
                              tf.TensorShape([BATCH_SIZE, 1])))
  optimizer = tf.train.GradientDescentOptimizer(LEARNING_RATE)
    
  model = Word2Vec(vocab_size=VOCAB_SIZE, embed_size=EMBED_SIZE)

  grad_fn = tfe.implicit_value_and_gradients(model.compute_loss)
  total_loss = 0.0  # for average loss in the last SKIP_STEP steps
  num_train_steps = 0
    
  while num_train_steps < NUM_TRAIN_STEPS:
    for center_words, target_words in tfe.Iterator(dataset):
      if num_train_steps >= NUM_TRAIN_STEPS:
        break
      loss_batch, grads = grad_fn(center_words, target_words)
      total_loss += loss_batch
      optimizer.apply_gradients(grads)
      if (num_train_steps + 1) % SKIP_STEP == 0:
        print('Average loss at step {}: {:5.1f}'.format(
                num_train_steps, total_loss / SKIP_STEP))
        total_loss = 0.0
      num_train_steps += 1

In [8]:
if __name__ == '__main__':
    main()

Successfully downloaded data/text8.zip
Average loss at step 4999:  65.6
Average loss at step 9999:  18.2
Average loss at step 14999:   9.6
Average loss at step 19999:   6.7
Average loss at step 24999:   5.7
Average loss at step 29999:   5.2
Average loss at step 34999:   5.0
Average loss at step 39999:   4.8
Average loss at step 44999:   4.8
Average loss at step 49999:   4.8
Average loss at step 54999:   4.8
Average loss at step 59999:   4.7
Average loss at step 64999:   4.7
Average loss at step 69999:   4.7
Average loss at step 74999:   4.6
Average loss at step 79999:   4.7
Average loss at step 84999:   4.7
Average loss at step 89999:   4.7
Average loss at step 94999:   4.6
Average loss at step 99999:   4.6
